#⚽ SoccerTrack Challenge 2025 Baseline Notebook

This notebook introduces a reference baseline for the SoccerTrack Challenge 2025.

## Overview

This baseline applies the Deep-EIoU multi-object tracking (MOT) github official code to the soccer match videos provided by the SoccerTrack Challenge 2025. The tracking results are evaluated using the ground truth annotations and the HOTA metric. Additionally, visualizations of the MOT output are provided for qualitative analysis.

![STC](https://drive.google.com/uc?export=view&id=1dpd9PZumzV3vjx47ZHyUJBg5KrYdkVIp)

## Pipeline Summary

1. **Tracking**  
   Run [Deep-EIoU](https://github.com/hsiangwei0903/Deep-EIoU) on the competition videos to generate tracking outputs in MOTChallenge format.

2. **Evaluation**  
   Evaluate the tracking performance using the provided ground truth and the [TrackEval](https://github.com/JonathonLuiten/TrackEval) toolkit, with HOTA as the main metric.

3. **Visualization**  
   Visualize the tracking results frame by frame to assist in interpreting tracking quality.

## In this notebook, you can learn:
1. Install Dependencies of Deep-EIoU
2. Download SoccerTrack Challenge 2025 Video Data
3. Run Tracking
4. Visualize Tracking Result
5. Install Dependencies of TrackEval
6. Preparation for HOTA Evaluation
7. Evaluate Results

# 1.Install Dependencies of Deep-EIoU

In [ ]:
# Install Python 3.7 and pip
!sudo apt-get install python3.7 python3.7-dev python3.7-distutils -y
!wget https://bootstrap.pypa.io/pip/3.7/get-pip.py
!python3.7 get-pip.py

# Install PyTorch (CUDA 11.6), and other dependencies
!python3.7 -m pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 \
torchaudio==0.13.0 --index-url https://download.pytorch.org/whl/cu116

!python3.7 -m pip install cython_bbox gdown \
https://github.com/KaiyangZhou/deep-person-reid/archive/master.zip

# Clone Deep-EIoU repo
!git clone https://github.com/hsiangwei0903/Deep-EIoU.git

# Install requirements for ReID
%cd /content/Deep-EIoU/Deep-EIoU/reid
!python3.7 -m pip install -r requirements.txt

# Go back to main directory
%cd /content/Deep-EIoU/Deep-EIoU

# Download the Detector and ReID model
!gdown --fuzzy 'https://drive.google.com/file/d/1834kh10-X0Tu743fgmN7jXPVDKgq4ZqR/view?usp=drive_link' --output checkpoints/best_ckpt.pth.tar
!gdown --fuzzy 'https://drive.google.com/file/d/14zzlm1nI9Ws_Il9RYNChwPC7Fsul7xwl/view?usp=drive_link' --output checkpoints/sports_model.pth.tar-60
# Download torchreid
!python3.7 -m pip install https://github.com/KaiyangZhou/deep-person-reid/archive/master.zip  # torchreid

# 2.Download SoccerTrack Challenge 2025 Video Data
The SoccerTrack Challenge 2025 datasets are available at https://drive.google.com/drive/folders/1_o78gcL4j0xHxbRjSR1Evs4VLXCr2ncD

In [ ]:
# Download the Video Data
# video 117092.mp4
%cd /content/Deep-EIoU/Deep-EIoU
gdown --fuzzy 'https://drive.google.com/file/d/1DQj5-kiU4VySymDXZuV84-nU1AZ4HZaj/view?usp=drive_link'

# 3.Run Tracking
<pre><code>python3.7 tools/demo.py --path &lt;video path&gt; </code></pre>
📍Outputs Path:
- Tracking result video:
/content/Deep-EIoU/Deep-EIoU/YOLOX_outputs/yolox_x_ch_sportsmot/track_vis/(timestamp)/117092.mp4
- Tracking result txt:
/content/Deep-EIoU/Deep-EIoU/YOLOX_outputs/yolox_x_ch_sportsmot/track_vis/*.txt

In [ ]:
# ▶️ Run tracking inference
!python3.7 tools/demo.py --path 117092.mp4 --save_result True

# 4.Visualize Tracking Result
The output videos from Deep-EIOU already include visualized tracking results. Additionally, we provide auxiliary code to visualize the tracking results from the MOT-format .txt files.

In [ ]:
import cv2
import os
import random

def load_mot_txt(txt_path):
    mot_data = {}
    with open(txt_path, 'r') as f:
        for line in f:
            frame_id, obj_id, x, y, w, h, *_ = line.strip().split(',')
            frame_id = int(frame_id)
            if frame_id not in mot_data:
                mot_data[frame_id] = []
            mot_data[frame_id].append((int(obj_id), float(x), float(y), float(w), float(h)))
    return mot_data

# ID Color
def get_color(idx):
    random.seed(idx)
    return tuple(random.randint(0, 255) for _ in range(3))  # RGB

def visualize_tracking_to_video(video_path, txt_path, output_video_path):
    cap = cv2.VideoCapture(video_path)
    mot_data = load_mot_txt(txt_path)
    frame_idx = 1

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Save video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx in mot_data:
            for tid, x, y, w, h in mot_data[frame_idx]:
                p1 = (int(x), int(y))
                p2 = (int(x + w), int(y + h))
                color = get_color(tid)
                cv2.rectangle(frame, p1, p2, color, 2)
                cv2.putText(frame, f'ID {tid}', (p1[0], p1[1]-5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print(f"Tracking result video saved to: {output_video_path}")

# Path
video_path = 'path/to/video.mp4'
txt_path = 'path/to/result.txt'
output_video_path = 'path/to/output_tracking.mp4'

visualize_tracking_to_video(video_path, txt_path, output_video_path)

# 5.Install Dependencies of TrackEval


In [ ]:
 #Clone TrackEval and install dependencies
%cd /content/
!git clone https://github.com/JonathonLuiten/TrackEval.git
!python3.7 -m pip install lap cython motmetrics
%cd /content/TrackEval/
!python3.7 -m pip install -r requirements.txt

# Prepare directories for GT and predictions
!mkdir -p /content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt
!mkdir -p /content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data

# 6.Preparation for HOTA Evaluation
When using the official code to compute HOTA, the related files must be organized according to the specified dataset format.
The following provides the relevant code; for details, please refer to the GitHub homepage [TrackEval](https://github.com/JonathonLuiten/TrackEval).

**Note: The frame index of GT txt file and tracking result txt file
must start from 1.**


In [ ]:
# Download GT file (ground truth)
!gdown --fuzzy 'https://drive.google.com/file/d/1cQeC4c0FG8i8Xayw3LdzhEU5p42s7zD1/view?usp=drive_link' \
--output /content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt/gt.txt

# Copy prediction results from Deep-EIoU
import glob, shutil, os
files_path = glob.glob('/content/Deep-EIoU/Deep-EIoU/YOLOX_outputs/yolox_x_ch_sportsmot/track_vis/*.txt')
for file in files_path:
    shutil.copy(file, "/content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data/117092.txt")

# Create seqmap (defines which sequences are used)
seqmap_path = "/content/TrackEval/data/gt/mot_challenge/mydataset/seqmaps/"
os.makedirs(seqmap_path, exist_ok=True)
with open(os.path.join(seqmap_path, "mydataset-test.txt"), "w") as f:
    f.write("name\n117092")

# Generate seqinfo.ini (basic video metadata)
ini_content = (
    "[Sequence]\n"
    "name=117092\n"
    "imDir=img1\n"
    "frameRate=30\n"
    "seqLength=1000000\n"
    "imWidth=1920\n"
    "imHeight=1080\n"
    "imExt=.jpg"
)
with open("/content/TrackEval/data/gt/mot_challenge/mydataset/117092/seqinfo.ini", "w") as f:
    f.write(ini_content)

# Format .txt files to comply with TrackEval (frame starts at 1, class ID = 1, space-separated)
def update_file_in_place(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    with open(file_path, 'w', encoding='utf-8') as f:
        for line in lines:
            parts = line.strip().split(',')
            if parts:
                if parts[0].isdigit():
                    parts[0] = str(int(parts[0]) + 1)
                if len(parts) >= 3:
                    parts[-3] = '1'  # Set class ID = 1
                f.write(' '.join(parts) + '\n')
            else:
                f.write(line)

# Apply formatting to GT and prediction result files
gt_txt = '/content/TrackEval/data/gt/mot_challenge/mydataset/117092/gt/gt.txt'
pred_txt = '/content/TrackEval/data/trackers/mot_challenge/mydataset/my_tracker/data/117092.txt'
update_file_in_place(gt_txt)
update_file_in_place(pred_txt)

# 7.Evaluate Results

In [ ]:
# Run evaluation
%cd /content/TrackEval
!python3.7 scripts/run_mot_challenge.py \
  --BENCHMARK mydataset \
  --SPLIT_TO_EVAL test \
  --GT_FOLDER data/gt/mot_challenge/mydataset \
  --TRACKERS_FOLDER data/trackers/mot_challenge/mydataset \
  --TRACKERS_TO_EVAL my_tracker \
  --METRICS HOTA CLEAR Identity \
  --SKIP_SPLIT_FOL True